<a href="https://colab.research.google.com/github/sabvz-run/solarium_siep/blob/main/notebooks/solarium_siep_principal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ☀️ Solarium SIEP
## Sistema Inteligente de Estimativa de Produtividade Solar

**Projeto:** A3 - 2025.2  
**Disciplina:** Sistemas de Controle e Inteligência Artificial  
**Curso:** Engenharia da Computação  
**Professor:** Noberto Maciel

**Equipe:**
- [Demerson Sampaio - 12725235937]
- [Josué Serra - 12724147248]
- [Rian Assis - 1272418628]
- [Sabrina Oliveira - 12725234634]

## Objetivos do Projeto
### Módulo 1: Predição de Geração
Desenvolver modelos de Machine Learning para prever a geração de energia solar (kWh) com alta precisão.

**Técnicas utilizadas:**
- Regressão Linear, Random Forest, Gradient Boosting
- Grid Search para otimização de hiperparâmetros
- Lógica Fuzzy para classificar eficiência dos painéis

### Módulo 2: Sistema de Recomendações
Criar sistema inteligente que recomenda:
- Quando armazenar energia na bateria
- Quando vender energia para a rede
- Quando economizar (previsão de baixa geração)
- Melhor momento para usar eletrodomésticos

---

## Datasets Utilizados

| # | Dataset | Período | Uso |
|---|---------|---------|-----|
| 1 | Geração 5 anos | 2015-2020 | Treinamento principal |
| 2 | Plant1 Industrial | 34 dias | Validação de alta resolução |
| 3 | NASA POWER | 2022-2024 | Validação externa |
| 4 | Geração + Consumo | Variável | Sistema de recomendações |

Detalhes: [/datasets/README.md](https://github.com/sabvz-run/solarium_siep/tree/main/datasets)

---
**Repositório GitHub:** https://github.com/sabvz-run/solarium_siep  
**Versão:** 1.0

In [9]:
# Instalação da biblioteca de Lógica Fuzzy
# É a única que não vem instalada por padrão no Colab
!pip install -q scikit-fuzzy

In [10]:
# ============================================================================
# IMPORTAÇÃO DE BIBLIOTECAS
# ============================================================================

# Manipulação de dados
import pandas as pd              # Trabalhar com DataFrames (tabelas)
import numpy as np               # Operações numéricas e arrays

# Visualização de dados
import matplotlib.pyplot as plt  # Criar gráficos
import seaborn as sns            # Gráficos estatísticos avançados

# Machine Learning - Modelos de Regressão
from sklearn.linear_model import LinearRegression              # Modelo baseline simples
from sklearn.ensemble import RandomForestRegressor             # Modelo ensemble robusto
from sklearn.ensemble import GradientBoostingRegressor         # Modelo avançado de boosting

# Machine Learning - Validação e Otimização
from sklearn.model_selection import train_test_split           # Dividir dados treino/teste
from sklearn.model_selection import GridSearchCV               # Busca de hiperparâmetros (requisito A3)

# Machine Learning - Pré-processamento
from sklearn.preprocessing import StandardScaler               # Normalizar features (melhora performance)

# Machine Learning - Métricas de Avaliação
from sklearn.metrics import mean_squared_error                 # Erro quadrático médio
from sklearn.metrics import mean_absolute_error                # Erro absoluto médio
from sklearn.metrics import r2_score                           # Coeficiente de determinação (0-1)

# Lógica Fuzzy (requisito do projeto)
import skfuzzy as fuzz                    # Funções fuzzy (pertinência, operações)
from skfuzzy import control as ctrl       # Sistema de controle fuzzy

# Utilidades
import warnings
warnings.filterwarnings('ignore')         # Suprimir avisos não críticos

# Configurações de visualização
plt.style.use('seaborn-v0_8-darkgrid')    # Estilo dos gráficos
sns.set_palette("husl")                    # Paleta de cores
plt.rcParams['figure.figsize'] = (12, 6)   # Tamanho padrão dos gráficos

print("Bibliotecas importadas com sucesso")

Bibliotecas importadas com sucesso


In [11]:
# ============================================================================
# CARREGAMENTO INTELIGENTE DE DATASETS
# ============================================================================
#
# ESTRATÉGIA DE FALLBACK (Redundância):
# 1. Tenta carregar do GitHub (público, rápido, sem autenticação)
# 2. Se falhar, carrega do Google Drive (backup confiável)
#
# VANTAGENS:
# - Não precisa upload manual durante apresentação
# - GitHub Raw permite acesso direto via URL
# - Drive como backup garante 100% de disponibilidade
# - Tratamento especial para NASA (formato problemático)
# ============================================================================

import requests
from io import StringIO

# Configuração de URLs
GITHUB_BASE = "https://raw.githubusercontent.com/sabvz-run/solarium_siep/main/datasets/"
DRIVE_PATH = "/content/drive/MyDrive/solarium_datasets/"

# Lista de datasets necessários
DATASETS = [
    'geracao_5anos.csv',
    'plant1_geracao.csv',
    'plant1_clima.csv',
    'nasa_clima.csv',
    'geracao_treino.csv',
    'geracao_teste.csv',
    'clima_treino.csv',
    'consumo_treino.csv',
    'consumo_teste.csv'
]


def carregar_do_github(arquivo):
    """
    Carrega dataset do GitHub usando HTTP GET.

    Como funciona:
    - requests.get() baixa o conteúdo do arquivo como texto
    - StringIO() converte texto em objeto file-like
    - pd.read_csv() lê o objeto como se fosse arquivo local

    Por que GitHub Raw?
    - URLs públicas, acessíveis sem autenticação
    - Respostas rápidas (CDN do GitHub)
    - Ideal para datasets < 100MB
    """
    try:
        url = GITHUB_BASE + arquivo
        response = requests.get(url, timeout=10)

        if response.status_code == 200:
            df = pd.read_csv(StringIO(response.text))
            return df
        else:
            return None
    except Exception as e:
        return None


def carregar_do_drive(arquivo):
    """
    Carrega dataset do Google Drive montado.

    Requisito: Drive já montado via drive.mount()

    Por que Drive?
    - Backup confiável se GitHub falhar
    - Acesso rápido a arquivos grandes
    - Controle total sobre versionamento
    """
    try:
        df = pd.read_csv(DRIVE_PATH + arquivo)
        return df
    except:
        return None


def carregar_nasa_especial(metodo='github'):
    """
    Tratamento especial para arquivo NASA.

    PROBLEMA IDENTIFICADO:
    - Arquivo NASA tem metadados nas primeiras linhas
    - Linha 12 tem erro de formatação (1 campo vs 7 esperados)
    - Header real pode estar em linha não padrão

    SOLUÇÃO:
    - skiprows: pula linhas problemáticas
    - on_bad_lines='skip': ignora linhas malformadas
    - engine='python': parser mais tolerante que C

    ALTERNATIVA: Se continuar falhando, tente:
    - skiprows=range(0,15) (pula mais linhas)
    - sep='\t' (testar separador tab)
    - encoding='latin-1' (encoding alternativo)
    """
    try:
        if metodo == 'github':
            url = GITHUB_BASE + 'nasa_clima.csv'
            response = requests.get(url, timeout=10)
            if response.status_code == 200:
                # Estratégia 1: Pular primeiras 10 linhas
                df = pd.read_csv(
                    StringIO(response.text),
                    skiprows=10,              # Ajustar se necessário
                    on_bad_lines='skip',      # Ignorar linhas ruins
                    engine='python'           # Parser mais robusto
                )
                return df
        else:
            df = pd.read_csv(
                DRIVE_PATH + 'nasa_clima.csv',
                skiprows=10,
                on_bad_lines='skip',
                engine='python'
            )
            return df
    except Exception as e:
        # Plano B: Tentar sem pular linhas
        try:
            if metodo == 'github':
                url = GITHUB_BASE + 'nasa_clima.csv'
                response = requests.get(url)
                df = pd.read_csv(
                    StringIO(response.text),
                    on_bad_lines='skip',
                    engine='python'
                )
                return df
        except:
            return None


# ============================================================================
# EXECUÇÃO
# ============================================================================

print("Iniciando carregamento de datasets...")
print("Estratégia: GitHub (primário) -> Drive (backup)\n")

# Montar Google Drive (necessário para fallback)
try:
    import os
    if not os.path.exists('/content/drive'):
        from google.colab import drive
        print("Montando Google Drive...")
        drive.mount('/content/drive')
        print("Drive montado!\n")
except:
    print("Aviso: Drive não disponível (apenas GitHub será usado)\n")

# Dicionário para armazenar datasets
datasets_loaded = {}

# Carregar cada dataset
for arquivo in DATASETS:
    nome = arquivo.replace('.csv', '')
    print(f"Carregando {arquivo}...", end=' ')

    # Tratamento especial para NASA
    if arquivo == 'nasa_clima.csv':
        df = carregar_nasa_especial('github')
        if df is None:
            df = carregar_nasa_especial('drive')
    else:
        # Padrão: tentar GitHub, depois Drive
        df = carregar_do_github(arquivo)
        if df is None:
            df = carregar_do_drive(arquivo)

    # Armazenar resultado
    if df is not None:
        datasets_loaded[nome] = df
        print(f"OK ({df.shape[0]} linhas x {df.shape[1]} colunas)")
    else:
        print("FALHA")

print(f"\nRESUMO: {len(datasets_loaded)}/{len(DATASETS)} datasets carregados")

# Criar variáveis globais para facilitar acesso
# (Em vez de datasets_loaded['geracao_5anos'], usamos df_geracao_5anos)
df_geracao_5anos = datasets_loaded.get('geracao_5anos')
df_plant1_geracao = datasets_loaded.get('plant1_geracao')
df_plant1_clima = datasets_loaded.get('plant1_clima')
df_nasa = datasets_loaded.get('nasa_clima')
df_geracao_treino = datasets_loaded.get('geracao_treino')
df_geracao_teste = datasets_loaded.get('geracao_teste')
df_clima_treino = datasets_loaded.get('clima_treino')
df_consumo_treino = datasets_loaded.get('consumo_treino')
df_consumo_teste = datasets_loaded.get('consumo_teste')

print("\nVariáveis globais criadas:")
variaveis = [
    ('df_geracao_5anos', df_geracao_5anos),
    ('df_plant1_geracao', df_plant1_geracao),
    ('df_plant1_clima', df_plant1_clima),
    ('df_nasa', df_nasa),
    ('df_geracao_treino', df_geracao_treino),
    ('df_geracao_teste', df_geracao_teste),
    ('df_clima_treino', df_clima_treino),
    ('df_consumo_treino', df_consumo_treino),
    ('df_consumo_teste', df_consumo_teste)
]

for nome, df in variaveis:
    if df is not None:
        print(f"  {nome}: {df.shape}")
    else:
        print(f"  {nome}: NÃO CARREGADO")

print("\nCarregamento concluído!")

Iniciando carregamento de datasets...
Estratégia: GitHub (primário) -> Drive (backup)

Montando Google Drive...
Aviso: Drive não disponível (apenas GitHub será usado)

Carregando geracao_5anos.csv... OK (364637 linhas x 2 colunas)
Carregando plant1_geracao.csv... OK (68778 linhas x 7 colunas)
Carregando plant1_clima.csv... OK (3182 linhas x 6 colunas)
Carregando nasa_clima.csv... OK (26305 linhas x 1 colunas)
Carregando geracao_treino.csv... OK (46704 linhas x 4 colunas)
Carregando geracao_teste.csv... OK (336 linhas x 4 colunas)
Carregando clima_treino.csv... OK (48408 linhas x 13 colunas)
Carregando consumo_treino.csv... OK (46704 linhas x 2 colunas)
Carregando consumo_teste.csv... OK (336 linhas x 2 colunas)

RESUMO: 9/9 datasets carregados

Variáveis globais criadas:
  df_geracao_5anos: (364637, 2)
  df_plant1_geracao: (68778, 7)
  df_plant1_clima: (3182, 6)
  df_nasa: (26305, 1)
  df_geracao_treino: (46704, 4)
  df_geracao_teste: (336, 4)
  df_clima_treino: (48408, 13)
  df_consum

In [13]:
# ============================================================================
# ANÁLISE EXPLORATÓRIA INICIAL - VISÃO GERAL DOS DATASETS
# ============================================================================
#
# O QUE ESTA CÉLULA FAZ:
# - Mostra estrutura de cada dataset (colunas, tipos, tamanho)
# - Identifica valores ausentes (precisamos tratar antes de modelar)
# - Exibe estatísticas descritivas (média, mediana, quartis)
# - Mostra primeiras linhas para entender formato dos dados
#
# POR QUE É IMPORTANTE:
# - Identificar problemas de qualidade de dados
# - Entender distribuições (detectar outliers)
# - Planejar pré-processamento necessário
# - Verificar se colunas esperadas existem
# ============================================================================

def explorar_dataset(df, nome):
    """
    Função para exploração padronizada de datasets.

    Reutilização de código (DRY - Don't Repeat Yourself):
    - Em vez de repetir o mesmo código 9 vezes,
    - Criamos uma função que recebe dataset e nome como parâmetros
    """
    if df is None:
        print(f"\n{nome}: Dataset não carregado\n")
        return

    print(f"\n{'='*70}")
    print(f"DATASET: {nome}")
    print(f"{'='*70}")

    # Informações básicas
    print(f"\nDimensões: {df.shape[0]:,} linhas x {df.shape[1]} colunas")
    print(f"\nColunas: {list(df.columns)}")

    # Tipos de dados (importante para conversões)
    # object = texto, int64 = inteiro, float64 = decimal
    print(f"\nTipos de dados:")
    print(df.dtypes)

    # Valores ausentes (NaN, None, vazios)
    # Precisamos decidir: preencher, remover ou interpolar?
    print(f"\nValores ausentes:")
    missing = df.isnull().sum()
    if missing.sum() > 0:
        print(missing[missing > 0])
    else:
        print("Nenhum valor ausente")

    # Primeiras linhas (verificar formato)
    print(f"\nPrimeiras 3 linhas:")
    display(df.head(3))

    # Estatísticas descritivas
    # count: quantidade de valores não nulos
    # mean: média aritmética
    # std: desvio padrão (dispersão)
    # min/max: valores extremos
    # 25%/50%/75%: quartis (percentis)
    print(f"\nEstatísticas descritivas:")
    display(df.describe())


# Explorar dataset principal (5 anos)
explorar_dataset(df_geracao_5anos, "Geração Solar (5 anos)")

# Explorar Plant1 (alta resolução)
explorar_dataset(df_plant1_geracao, "Plant1 - Geração")
explorar_dataset(df_plant1_clima, "Plant1 - Clima")

# Explorar NASA (validação externa)
explorar_dataset(df_nasa, "NASA POWER")

# Explorar datasets de treino/teste
explorar_dataset(df_geracao_treino, "Geração - Treino")
explorar_dataset(df_clima_treino, "Clima - Treino")
explorar_dataset(df_consumo_treino, "Consumo - Treino")

print("\n" + "="*70)
print("EXPLORAÇÃO INICIAL CONCLUÍDA")
print("="*70)
print("\nPRÓXIMOS PASSOS:")
print("1. Tratamento de valores ausentes")
print("2. Conversão de datas para datetime")
print("3. Feature engineering (criar variáveis temporais)")
print("4. Visualização de padrões e correlações")


DATASET: Geração Solar (5 anos)

Dimensões: 364,637 linhas x 2 colunas

Colunas: ['1587692809.0', '0.0']

Tipos de dados:
1587692809.0    float64
0.0             float64
dtype: object

Valores ausentes:
Nenhum valor ausente

Primeiras 3 linhas:


,1587692809.0,0.0
0,1587693109.00,0.00
1,1587693409.00,0.00
2,1587693709.00,0.00



Estatísticas descritivas:


,1587692809.0,0.0
count,364637.00,364637.00
mean,1642388509.00,1.00
std,31578533.82,1.80
min,1587693109.00,0.00
25%,1615040809.00,0.00
50%,1642388509.00,0.13
75%,1669736209.00,1.14
max,1697083909.00,168.20



DATASET: Plant1 - Geração

Dimensões: 68,778 linhas x 7 colunas

Colunas: ['DATE_TIME', 'PLANT_ID', 'SOURCE_KEY', 'DC_POWER', 'AC_POWER', 'DAILY_YIELD', 'TOTAL_YIELD']

Tipos de dados:
DATE_TIME       object
PLANT_ID         int64
SOURCE_KEY      object
DC_POWER       float64
AC_POWER       float64
DAILY_YIELD    float64
TOTAL_YIELD    float64
dtype: object

Valores ausentes:
Nenhum valor ausente

Primeiras 3 linhas:


...


DATASET: Consumo - Treino

Dimensões: 46,704 linhas x 2 colunas

Colunas: ['datetime', 'demand_MW']

Tipos de dados:
datetime      object
demand_MW    float64
dtype: object

Valores ausentes:
Nenhum valor ausente

Primeiras 3 linhas:


,datetime,demand_MW
0,2017-11-03 00:00:00,2.19
1,2017-11-03 00:30:00,2.14
2,2017-11-03 01:00:00,2.01



Estatísticas descritivas:


,demand_MW
count,46704.00
mean,2.81
std,0.90
min,0.00
25%,2.12
50%,2.71
75%,3.42
max,8.11



EXPLORAÇÃO INICIAL CONCLUÍDA

PRÓXIMOS PASSOS:
1. Tratamento de valores ausentes
2. Conversão de datas para datetime
3. Feature engineering (criar variáveis temporais)
4. Visualização de padrões e correlações
